# Calculate Statistics

## Imports

In [1]:
# Managing data
import pandas as pd
# DB connection
from scraping import create_connection
# Files & I/O
import pickle
import csv
import os
from pathlib import Path
# Plotting
import matplotlib.pyplot as plt

## Load Data

In [2]:
p = Path.cwd()
path_parent = p.parents[1]

In [3]:
# database
path_db = str(path_parent / "database" / "netmums-merged.db")
path_clean_data = path_parent / "clean_data" / "netmums"


In [8]:
sql = """
    SELECT
        DISTINCT p.user_url
    FROM posts AS p
    LEFT JOIN threads AS t
    ON t.id=p.thread_id
    LEFT JOIN subforums AS s
    ON s.id=t.subforum_id
    WHERE s.forum_id=24
    AND p.user_url<>"Anonymous"
"""
# sql = """
# SELECT *
# FROM posts
# LIMIT 2
# """
# forum_id = 24

In [9]:
conn = create_connection(path_db)
df = pd.read_sql_query(sql, conn)
conn.close()

In [10]:
df

,id,thread_id,post_count,post_id,user_url,date_created,date_recorded,body,version
0,1,1,1,19709335,sarah-l-2389,2021-01-28 03:39PM,2021-02-26 20:58:01,Once again we've teamed up with Product of the...,1
1,2,1,2,19709336,wendy-f-156,2021-01-28 07:07PM,2021-02-26 20:58:01,Nando's Garlic PERinaisse PERi-PERi Mayonnaise...,1


## text length

In [13]:
sql = """
    SELECT
        AVG(LENGTH(text_clean))
	FROM text
"""
with create_connection(path_db) as conn:
    df = pd.read_sql_query(sql, conn)
df

,AVG(LENGTH(text_clean))
0,386.199554


In [6]:
sql = """
SELECT LENGTH(text_clean)
FROM text
ORDER BY LENGTH(text_clean)
LIMIT 1
OFFSET (SELECT COUNT(*)
        FROM text) / 2
"""
with create_connection(path_db) as conn:
    df = pd.read_sql_query(sql, conn)
df

,LENGTH(text_clean)
0,240


In [9]:
sql = """
    SELECT
        count(*)
	FROM text
    LEFT JOIN posts AS p
    ON text.post_id = p.id
    LEFT JOIN threads AS t
    ON t.id=p.thread_id
    LEFT JOIN subforums AS s
    ON s.id=t.subforum_id
    WHERE s.forum_id=24
"""
with create_connection(path_db) as conn:
    df = pd.read_sql_query(sql, conn)
df

,count(*)
0,210308


In [8]:
sql = """
    SELECT
        count(*)
	FROM text
    LEFT JOIN posts AS p
    ON text.post_id = p.id
    LEFT JOIN threads AS t
    ON t.id=p.thread_id
    LEFT JOIN subforums AS s
    ON s.id=t.subforum_id
    WHERE s.forum_id=24
    AND LENGTH(text_clean) < 30
"""
with create_connection(path_db) as conn:
    df = pd.read_sql_query(sql, conn)
df

,count(*)
0,4229


In [12]:
210308 / 15000000

0.014020533333333333

In [ ]:
sql = """
with concat_threads as (
SELECT
    p.thread_id AS thread_id,
    group_concat(text.text_clean, ' ') AS text_clean
FROM text
LEFT JOIN posts AS p
ON text.post_id = p.id
LEFT JOIN threads AS t
group by p.thread_id
)
select AVG(LENGTH(text_clean))
from concat_threads
"""
with create_connection(path_db) as conn:
    df = pd.read_sql_query(sql, conn)
df

In [4]:
sql = """
SELECT text_clean
FROM text
ORDER BY LENGTH(text_clean)
LIMIT 20
"""
with create_connection(path_db) as conn:
    df = pd.read_sql_query(sql, conn)
df

,text_clean
0,F
1,x
2,M
3,x
4,?
5,.
6,?
7,y
8,?
9,X
